In [20]:
import pandas as pd 
import ast as ast
import re

In [21]:
df_games = pd.read_parquet('./data_clean_EDA/steam_games.parquet')

In [22]:

df_games = df_games.drop(columns=['publisher', 'title','tags', 'url', 'reviews_url', 'specs', 'early_access'])

In [23]:
# Asegurarse de que 'genres' es una cadena
df_games['genres'] = df_games['genres'].astype(str)

# Función para eliminar "Free to Play" de los géneros
def eliminar_free_to_play(generos):
    # Usar una expresión regular para separar los géneros por palabras en mayúscula
    lista_generos = re.findall(r'[A-Z][a-z]*', generos)  # Encuentra palabras que comienzan con mayúscula
    # Eliminar "Free to Play" si está presente
    lista_generos = [g for g in lista_generos if g != "Free" and g != "Play"]
    # Convertir de nuevo a cadena y devolver
    return ', '.join(lista_generos) if lista_generos else None

# Aplicar la función a la columna 'genres'
df_games['genres'] = df_games['genres'].apply(eliminar_free_to_play)

# Extraer el primer género
df_games['genres'] = df_games['genres'].str.split(', ').str[0]
df_games['id'] = df_games['id'].astype(str)


In [24]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        32115 non-null  object 
 1   app_name      32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   price         30758 non-null  float64
 4   id            32135 non-null  object 
 5   developer     28836 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


In [26]:
# divide generos 
genres_dummies = (
    df_games['genres'].explode()
    .str.get_dummies()
    .groupby(level=0)
    .sum()
    .reset_index(drop=True)
)

df_generos = pd.concat([df_games[['id']], genres_dummies], axis=1)
keep_cols = df_generos.columns[~df_generos.columns.str.contains('&amp;', case=False)]
df_generos = df_generos[keep_cols]
df_generos.columns = pd.unique(df_generos.columns)
df_generos['id'] = df_generos['id'].astype(str)

__Guardaremos los datas ya trabajados para el modelo de recomendacion__

In [27]:
df_generos.to_parquet('./data_clean_EDA/df_dummies.parquet', index=False)

In [25]:
df_games.to_parquet('./data_clean_EDA/df_games_ML.parquet', index=False)